In [ ]:
import json 
import os
import collections
import tensorflow as tf 
import re
import h5py
import argparse
import sys 
import numpy as np 
import pandas as pd
import pickle
from nltk.translate.bleu_score import corpus_bleu

LOAD PREPROCESSED TRAINING DATASET

In [ ]:
#Load necessary files: Here we have loaded file for training
df_final = pickle.load(open('train_df_final.pkl', 'rb'))
features = pickle.load(open('image_feature_train.pkl', 'rb'))
word_to_idx = pickle.load(open('word_to_idx.pkl', 'rb'))
idx_to_word = pickle.load(open('idx_to_word.pkl', 'rb'))

In [ ]:
from keras.layers import Input, Embedding, merge, LSTM, Dropout, Dense, RepeatVector, BatchNormalization, \
    TimeDistributed, Flatten, Reshape
from keras.models import Model, model_from_json
from keras.optimizers import Adam
from keras import optimizers

# Training

In [ ]:
vocabulary_size=len(word_to_idx)
embed_size=150
question_max_len=PADDING_LEN-1
answer_max_len=PADDING_LEN-1
# Image
k_image_input = Input(shape=(4096,))
k_image_repeat = RepeatVector(n=question_max_len)(k_image_input)

# Question
k_question_input = Input(shape=(question_max_len,), dtype='int32')
k_question_embedded = Embedding(input_dim=vocabulary_size, output_dim=embed_size, input_length=question_max_len)(k_question_input)  
k_question_embedded = Dropout(0.3)(k_question_embedded)

# Merge
k_merged = merge([k_image_repeat, k_question_embedded], mode='concat')  # Merge for layers merge for tensors
k_merged = BatchNormalization()(k_merged)

#Set encoder
k_encoder_outputs, k_state_h, k_state_c  = LSTM(embed_size, return_state=True)(k_merged)
k_encoder_outputs = Dropout(0.3)(k_encoder_outputs)
k_encoder_states = [k_state_h, k_state_c]

# Set up the decoder, using `encoder_states` as initial state.
k_decoder_inputs = Input(shape=(None, vocabulary_size))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
k_decoder_lstm = LSTM(embed_size, return_sequences=True, return_state=True)
k_decoder_outputs, _, _ = k_decoder_lstm(k_decoder_inputs, initial_state=k_encoder_states)

#Final Layer
k_decoder_dense = Dense(vocabulary_size, activation='softmax')
k_decoder_outputs = k_decoder_dense(k_decoder_outputs)

model = Model([k_image_input, k_question_input, k_decoder_inputs], k_decoder_outputs)
print('Model created')
print('Compiling model...')
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
print('Model compiled')

In [ ]:
image_input=np.array(features)
question_inputs=np.array(df_final['Q_Encoded'])
question_inputs=np.array([np.array(item) for item in question_inputs])
decoder_inputs =np.array(df_final['A_Encoded'])
decoder_targets =np.array(df_final['A_Encoded'])

In [ ]:
m_decoder_inputs = np.zeros(
    (len(decoder_inputs), PADDING_LEN-1, vocabulary_size),
    dtype='float32')
m_decoder_targets= np.zeros(
    (len(decoder_targets), PADDING_LEN-1, vocabulary_size),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(question_inputs, decoder_inputs)):
    for t, char in enumerate(input_text):
        m_question_input_data[i, t, char] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        m_decoder_inputs[i, t, char] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            m_decoder_targets[i, t - 1, char] = 1.

In [ ]:
model.fit([features,question_inputs, m_decoder_inputs], m_decoder_targets, batch_size=100, epochs=100, validation_split=0.2)


# Inference

In [ ]:
encoder_model = Model([k_image_input,k_question_input], k_encoder_states)
decoder_state_input_h = Input(shape=(embed_size,))
decoder_state_input_c = Input(shape=(embed_size,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = k_decoder_lstm(k_decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = k_decoder_dense(decoder_outputs)
decoder_model = Model([k_decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)

In [ ]:
def decode_sequence(image_features,input_seq):
    # Encode the input as state vectors.
  
    states_value = encoder_model.predict([image_features,input_seq])

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, vocabulary_size))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, 1] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx_to_word[sampled_token_index]
        decoded_sentence += sampled_char
        decoded_sentence +=" "

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '2' or
           len(decoded_sentence) > PADDING_LEN-1):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, vocabulary_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

LOAD PREPROCESSED TEST DATASET

In [ ]:
#Open inputs of test dataset
file = open('image_feature_test.pkl', 'rb')
features_test = pickle.load(file)
file2 = open('test_df_v_final', 'rb')
df_test= pickle.load(file2)

In [ ]:
test_questions=np.array(df_test['Q_Encoded'])
test_questions=np.array([np.array(item) for item in test_questions])

In [ ]:
#use model to predict the answer
predicted_answers_test=[]
for seq_index in range(len(df_test)):
    input_seq = test_questions[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(features_test[seq_index: seq_index + 1], input_seq)
    predicted_answers_test.append(decoded_sentence.split(' '))

In [ ]:
predicted_answers_test=[item[0:len(item)-1] for item in predicted_answers_test]

In [ ]:
#function used to remove end tokens from the predicted answers
def find_end(item,st):
    for i in range(0,len(item)):
        if item[i]==st:
            return i
    return i+1  
predicted_answers_test=[item[0:find_end(item,'<END>')] for item in predicted_answers_test]

In [ ]:
#actual/target answers
test_answers=np.array(df_test['A_parsed'])
test_answers=[item for item in test_answers]


In [ ]:
#Final BLEU Score
score = corpus_bleu(test_answers, predicted_answers_test)
print(score)